<a href="https://colab.research.google.com/github/maquico/IA-IDS330/blob/main/ids330l_asignacion03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IDS330 | Asignación 03

**Objetivo**: Desarrollar una red neuronal densa para clasificar imagenes del dataset CIFAR10 con una precisión de al menos 40%.

**¿Cómo lo hago?**: *Pueden lograrlo como deseen*, pero pueden cambiar algunos de estos hiperparametros:
* Tamaño de la red (cantidad de capas o cantidad de neuronas por capas)
* Taza de aprendizaje (`learning rate`)
* Tamaño del mini-batch (`batch_size`)
* Cantidad de iteraciones (`epochs`)

In [ ]:
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
labels = ["airplane", 'auto', 'bird', 'cat', 'deer',
          'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
print(f"x_train: {x_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"x_test: {x_test.shape}")
print(f"y_test: {y_test.shape}")

In [ ]:
n = 16
fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(6,6))
axs = axs.ravel()
for i in range(16):
    axs[i].imshow(x_train[i])
    label = labels[y_train[i][0]]
    axs[i].set_title(label)
    axs[i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Incrementar el numero de capas
# Incrementar numero de neuronas en capas ocultas

class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.linear = torch.nn.Sequential(
            torch.nn.Linear(32*32*3, 128),
            torch.nn.Sigmoid(),
            torch.nn.Linear(128, 10),
            torch.nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.reshape(-1, int(32*32*3))
        x = torch.tensor(x).float()
        return self.linear(x)


In [ ]:
# Pueden jugar con los hiperparametros (learning rate, epochs, batch_size)
######
lr = 0.001
epochs = 10
batch_size = 64
######

device = "cuda" if torch.cuda.is_available() else 'cpu'

# Definiendo el modelo
model = NeuralNetwork()

loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

errors = []

# cuando queremos entrenar, debemos poner el modelo en modo entrenamiento
model.train()
model.cuda()

for i in range(epochs):
    error_per_batch = []
    for batch in range(x_train.shape[0]//batch_size - 1):

        # Batch
        start = batch * batch_size
        end = (batch+1) * batch_size
        x = x_train[start:end] # x_train[0:64], x_train[64:128], ...
        y = y_train[start:end]

        x = torch.tensor(x).float()
        y = torch.tensor(y).long()
        y = y.squeeze()

        x = x.to(device)  # x.cuda()
        y = y.to(device)  #

        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output, y)
        loss.backward()  # propagar el error
        optimizer.step() # actualizar los parametros

        error_per_batch.append(loss.item())
    errors.append(np.mean(error_per_batch))

In [ ]:
n = 16
fig, axs = plt.subplots(nrows=4, ncols=4, figsize=(10,10))
axs = axs.ravel()
for i in range(n):
    x = x_train[i]

    # feedfoward
    x = torch.tensor(x).float()
    output = model(x.cuda())
    prob, preds = torch.max(output, axis=1)

    label = labels[preds.item()]

    axs[i].imshow(x_train[i], cmap='gray')
    axs[i].set_title(f"Pred: {label}, prob: {prob.item():.2f}")
    axs[i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# cuando queremos entrenar, debemos poner el modelo en modo entrenamiento
model.eval()
accs=  []
with torch.no_grad():
    for batch in range(x_test.shape[0]//batch_size - 1):

        # Batch
        start = batch * batch_size
        end = (batch+1) * batch_size
        x = x_test[start:end] # x_train[0:64], x_train[64:128], ...
        y = y_test[start:end]

        x = torch.tensor(x).float()
        y = torch.tensor(y).long()
        x = x.cuda()
        y = y.cuda()
        y = y.squeeze()

        output = model(x)

        prob, preds = torch.max(output, axis=1)
        accuracy = torch.sum(y == preds) / y.shape[0]
        accs.append(accuracy.item())

In [ ]:
print(f"Accuracy: {np.mean(accs)*100:.2f}%")